# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \frac{1}{2|U|} \sum_{u \in U} \min_{x \in V} d(u, x) + \frac{1}{2|V|}\sum_{v \in V} \min_{x \in U} d(v, x) $$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{tp}$

$$ H^\textsf{pt}_d(U, V) = \max \left\{
        \max_{u \in U} \left\{\textsf{pt}(u, U) \min_{x \in V} d(u, x)\right\},
        \max_{v \in V} \left\{\textsf{pt}(v, V) \min_{x \in U} d(v, x)\right\}
     \right\} $$

# Notas adicionales
La versión de $H^\textsf{pt}_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.13"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Word2Vec"),
    PackageSpec(name="Embeddings"),
])

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, CSV, DataFrames, Base64, Random
using Embeddings: load_embeddings, FastText_Text
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.2 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.12.7 `../../Research/TextSearch.jl`
  [c64b6f0f] Word2Vec v0.5.3


In [3]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 2000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

In [4]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=true, group_url=true, group_num=true, del_diac=true, del_punc=true)

filename = "../data/spanish-news-in-twitter.csv"
embfile = filename * ".w2v.vec"
@time D = CSV.read(filename, DataFrame; delim=',', missingstring="XXXX")
@time D[:, :text] = String.(base64decode.(D.text))
rm(embfile)
word2vec_embeddings(textconfig, D.text, embfile; dim=64, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)


 20.064970 seconds (33.39 M allocations: 2.553 GiB, 5.32% gc time, 99.84% compilation time)
  0.960138 seconds (2.59 M allocations: 303.930 MiB, 7.98% gc time, 86.34% compilation time)
Starting training using file /home/sadit/IR-2022/Unidades/w2v.tmp
Vocab size: 12135
Words in train file: 442523


Embeddings.EmbeddingTable{Matrix{Float32}, Vector{String}}(Float32[0.006254 0.408892 … -0.239267 -0.161472; 0.006905 -1.695412 … 0.777975 0.751166; … ; -0.003277 1.584423 … -0.032014 -0.018186; 0.003372 -0.658217 … 0.066842 0.010714], ["</s>", "ha", "sus", "presidente", "contra", "anos", "fue", "tras", "pero", "gobierno"  …  "#ondasdelpodcast", "🤢", "#premiosortegaygasset", "infiel", "indigenista", "contenedores", "#cerocahui", "chueco", "yrma", "lydya"])

In [5]:
T = let corpus=D.text
    voc = Vocabulary(textconfig, corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

(textconfig = TextConfig(true, false, true, true, true, true, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=12135, train-n=41999, maxoccs=1987}, voc = Vocabulary(["</s>", "ha", "sus", "presidente", "contra", "anos", "fue", "tras", "pero", "gobierno"  …  "#ondasdelpodcast", "🤢", "#premiosortegaygasset", "infiel", "indigenista", "contenedores", "#cerocahui", "chueco", "yrma", "lydya"], Int32[0, 1987, 1953, 1926, 1954, 1956, 1614, 1411, 1361, 1359  …  5, 6, 5, 5, 6, 6, 5, 5, 6, 6], Int32[0, 1987, 1953, 1926, 1954, 1956, 1614, 1411, 1361, 1359  …  5, 6, 5, 5, 6, 6, 5, 5, 6, 6], Dict{String, UInt32}("focos" => 0x0000225f, "#videos" => 0x00001754, "libertad" => 0x000001be, "ayudas" => 0x00000f6f, "henry" => 0x00001fb4, "islamico" => 0x0000163b, "directivos" => 0x00001625, "soporte" => 0x000013bb, "astronautas" => 0x00002584, "advirtieron" => 0x0000234f…), 41999)

In [6]:
cloud(text, E, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, E, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

cloud_corpus (generic function with 1 method)

In [7]:
struct AWMD <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dcos::NormalizedCosineDistance
end

AWMD(X) = AWMD(X, [KnnResult(1) for _ in 1:Threads.nthreads()], NormalizedCosineDistance())

const PI_2 = convert(Float32, pi / 2)


@inline function nn_(X, res, dcos, u, V)
    res = reuse!(res)

    for (vid, _) in V
        push!(res, vid, evaluate(dcos, u, X[vid]))
    end
    
    minimum(res)
end

function onesidewmd(X, res, dcos, U, V)
    d::Float32 = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(X, res, dcos, X[uid], V))
        end
    end
    
    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::T, V::T) where T
    length(U) == 1 && haskey(U, 0) && return PI_2
    length(V) == 1 && haskey(V, 0) && return PI_2
    res = wmd.results[Threads.threadid()]
    dcos = wmd.dcos

    max(onesidewmd(wmd.X, res, dcos, U, V), onesidewmd(wmd.X, res, dcos, V, U))
end


In [8]:
db = VectorDatabase(cloud_corpus(D.text, E, T))
for c in eachcol(E.embeddings)
    normalize!(c)
end

dist = AWMD(MatrixDatabase(E.embeddings))

AWMD(MatrixDatabase{Matrix{Float32}}(Float32[0.17953031 0.026101155 … -0.10653875 -0.07246646; 0.19821823 -0.1082247 … 0.34641 0.3371132; … ; -0.09407112 0.101139836 … -0.014254918 -0.008161632; 0.09679824 -0.042016536 … 0.029762829 0.0048082992]), KnnResult[KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)  …  KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)], NormalizedCosineDistance())

In [9]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [10]:
index = ParallelExhaustiveSearch(; db, dist);

In [11]:
Threads.nthreads()

64

In [12]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [13]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = cloud(qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [16]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "calentamiento global", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.088636 seconds (166.64 k allocations: 3.814 MiB)


1,16524,0.072,azucenau,"La jefa de gobierno de la Ciudad de México, Claudia Sheinbaum, aseguró que ella y el presidente de la República, Andrés Manuel López Obrador, “somos parte del mismo movimiento”, y que jamás será líder de oposición, pues “somos parte de la Cuarta Transformación”. https://t.co/FYoh0GuROr"
2,4111,0.092,PedroFerriz,RT @CentralFM_: #CentralFmEquilibrio Estados Unidos urgió al Gobierno de Andrés Manuel López Obrador a reanudar la autorización de import…
3,41283,0.096,Reforma,"El Presidente Andrés Manuel López Obrador propuso al Gobierno de #NuevoLeon acordar con empresarios que produzcan menos, usen menos #agua, y que así vaya más líquido a la gente. https://t.co/9kW1cPZBmj"
4,35523,0.111,JoseCardenas1,"El Presidente Andrés Manuel López Obrador arribó esta tarde al Aeropuerto de La Habana, Cuba, para comenzar su gira de trabajo y sostener encuentros con el mandatario de la isla, Miguel Díaz-Canel https://t.co/GiBHboPAIG"
5,20587,0.124,SinEmbargoMX,"🚨 #URGENTE | El Gobierno de EU ofrece 5 millones de dólares por información que conduzca al arresto y condena de los mexicanos Ovidio Guzmán López, Iván Archivaldo Guzmán Salazar, Jesús Alfredo Guzmán Salazar y Joaquín Guzmán López, hijos de “El Chapo” https://t.co/S8eyJEDNTX https://t.co/YSXPkhUM7k"
6,20882,0.124,CiroGomezL,"El gobierno de #EU publicó los carteles de búsqueda de los 4 hijos de ""El Chapo"", en donde ofrecen hasta 5 millones de dólares por Joaquín Guzmán López, Ovidio Guzmán López, Iván Archivaldo Guzmán Salazar y Jesús Alfredo Guzmán Salazar: https://t.co/hMfarnfA6f"
7,21911,0.136,RicardoAlemanMx,"Si tenían duda, el @TEPJF_informa también está sometido por el gobierno de @lopezobrador_ ! Por eso hoy voto contra la decisión del @INEMexico ! Tienen razón Vargas Llosa y Ernesto Zedillo; con López, México vive la destrucción de su democracia! Al tiempo!"


  0.062425 seconds (166.64 k allocations: 3.814 MiB)


1,1361,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/jbAcSdIuPI"
2,32852,0.038,epigmenioibarra,"29 JoséE 30 JoséLuisL 31 Jhosivani 32 JulioCésarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa90Meses https://t.co/oFlt1F32LI"
3,31559,0.038,epigmenioibarra,"29 JoséE 30 JoséLuisL 31 Jhosivani 32 JulioCésarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa90Meses https://t.co/43JRhtAr00"
4,19770,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa86meses https://t.co/yQV8oABngZ"
5,2113,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/BNAdU3bp0n"
6,5391,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/cxi6tlCTMA"
7,18648,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa85meses https://t.co/KH0xDLkhWz"


  0.084231 seconds (166.64 k allocations: 3.814 MiB)


1,17410,0.046,TelemundoNews,"El acuerdo incluye promesas clave, entre ellas la de reducir la energía del carbón y las subvenciones a los combustibles fósiles, pero muchos críticos dicen que no ofrece suficiente apoyo a las naciones en desarrollo afectadas por el calentamiento global. https://t.co/foBCgxRNVT"
2,7442,0.052,patriciajaniot,"El calentamiento global es más rápido y grave de lo pensado. Unos diez años antes de lo estimado, se puede alcanzar el umbral de un incremento de la temperatura en 34.7 °F (1.5 ºC), con riesgos de desastres ""sin precedentes"" para la humanidad https://t.co/iQVOG3eVyZ"
3,10963,0.056,SinEmbargoMX,"#Opinión | Ana Ortega de @THL_Latino ""El consumo de carne o lo que no se habla del calentamiento global"" Si no adoptamos una postura más firme ante el calentamiento global, el panorama de México está a punto de ponerse sombrío https://t.co/HL8v7EBp2x https://t.co/k8Az8hDr5y"
4,11613,0.06,EFEnoticias,"El Reino Unido presionó al resto de Gobiernos para que anuncien más medidas contra el calentamiento global, apenas cinco semanas antes de la próxima cumbre del clima (COP26) que acogerá su país. https://t.co/bmKjR0hXRm"
5,7766,0.079,CNNEE,"▶ Todavía podemos mitigar el daño al planeta por el calentamiento global si exigimos políticas públicas a gobiernos, dicen expertas. https://t.co/7eoc24sboY"
6,15809,0.096,bbcmundo,Cambio climático: 5 buenas noticias sobre la lucha contra el calentamiento global (contadas por científicos y expertos) https://t.co/VOKUupHVGN
7,7410,0.099,AFPespanol,"#ÚLTIMAHORA El calentamiento global llegará a +1,5 °C hacia 2030, 10 años antes de lo previsto (ONU) #AFP https://t.co/KJT1ESxU2S"


  0.068182 seconds (166.64 k allocations: 3.814 MiB)


1,4169,0.121,AristeguiOnline,Castillo con coronavirus como corona rompe récord mundial Guinness | #Video https://t.co/SaJIKYierX https://t.co/FmV7YOWKEX
2,31951,0.121,EFEnoticias,"Carlo Ancelotti, técnico italiano del Real Madrid, sigue dando positivo en coronavirus y no integra la expedición a Londres. #UCL https://t.co/m8dp8nJVVu"
3,21053,0.13,abc_es,"🔴 #ÚLTIMAHORA Don Juan Carlos da negativo en la PCR, tras el positivo en Covid de Rafa Nadal https://t.co/P9Li2qJubQ"
4,3004,0.133,EFEnoticias,"RT @EFEeconomia: 🗓️Algunas claves económicas para este lunes: ▶️Comienza en Barcelona el primer MWC pos-Covid, con menos asistentes y sona…"
5,22797,0.133,el_pais,"Bruno Polo, un estudiante de Barcelona, se infectó de coronavirus por tercera vez estas Navidades. Los expertos admiten que casos como el suyo irán creciendo por la pérdida de efectividad de los anticuerpos con el paso del tiempo https://t.co/z6EERJl5iw"
6,23610,0.135,AFPespanol,"🎾 .@DjokerNole, expulsado de Australia tras una batalla judicial sobre su situación en cuanto a la vacuna contra el #covid19, es esperado en Europa después de pasar por Dubái en tránsito, tras el inicio del Open de Australia que soñaba ganar por décima vez #AFP https://t.co/COYmevydpF"
7,5969,0.14,AFPespanol,"El covid-19 volvió a acaparar protagonismo en los #JuegosOlímpicos: después de aplazar un año @Tokyo2020es y dejar afuera a varios participantes, fue más lejos y se llevó a dos estrellas, los golfistas Jon Rahm, de España, y Bryson DeChambeau, de EEUU #AFP #AFPdeportes ⛳🤒 https://t.co/n3mfS66Okw"


In [18]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.041952 seconds (166.64 k allocations: 3.814 MiB)


1,2818,0.0,Adela_Micha,Ya son 2 millones 498 mil 357 casos positivos en el país. https://t.co/d2RUSlyB57
2,34105,0.078,el_pais,Ya son 57 millones los estadounidenses (de un total de 331) los que usan el español como lengua de comunicación habitual o lo entienden correctamente. Este incremento está teniendo un claro reflejo en la Administración del país https://t.co/pKmhkL78ji
3,31567,0.084,SinEmbargoMX,"El Gobierno federal anunció nuevas medidas para combatir la crisis forense en el país, ya que en México existen casi 100 mil casos de personas desaparecidas https://t.co/dgR9Qp9SY1 https://t.co/OPiVewLzY2"
4,28961,0.094,azucenau,"#AzucenaALas10 | No se tiene una certeza de la cantidad de personas que han fallecido en #Ucrania o el número de pobladores que lograron salir... lo que sí sabemos, es que un pequeño niño de solo 11 años tuvo que viajar solo para escapar de la guerra https://t.co/a0a9XpN1oX"
5,28202,0.094,dw_espanol,"Un pueblo desafía a invasores #rusos La población civil de #Ucrania está mostrando una feroz resistencia al avance de las tropas rusas, aun cuando sus efectivos son minoría en cantidad y tecnología. Este pueblo se enfrentó al avance de un tanque. 👇 /ft #DWNoticias https://t.co/tloLFPhEeN"
6,21986,0.094,PublimetroMX,"#PublimetroMX abordó una de las ambulancias de la @CruzRoja_CDMX, en donde se constató cómo este tipo de unidades circulan de forma constante entre vialidades primarias o secundarias. https://t.co/wdV5A9v3la https://t.co/6L3zZXzI2F"
7,34002,0.095,Milenio,"#CadenaDeMando | Es muy temprano para juzgar a las actuales Fuerzas Armadas. Lo que es un hecho es que este país, en definitiva, sería un caos total sin la participación de los soldados de las tres fuerzas 🖊 Lee la columna de @elibarrola https://t.co/rCg9CjzqcS https://t.co/sZ6OelxdmA"


  0.104251 seconds (166.64 k allocations: 3.814 MiB)


1,22242,0.0,CiroGomezL,"#EnVivo en @Radio_Formula 104.1 fm, 1500AM y Telefórmula. Sigue aquí la transmisión en vivo: https://t.co/YtTR04mr07"
2,23026,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da clic aquí para vernos #EnVivo: https://t.co/lcQr4FeQHG https://t.co/wpLUFYxTiY
3,11910,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da click aquí para vernos #EnVivo: https://t.co/ewEXzfqnRn
4,12608,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da click aquí para vernos #EnVivo: https://t.co/t7iFAWuHQz
5,31677,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da clic aquí para vernos #EnVivo: https://t.co/D5HpOAjTWt https://t.co/C8jcDzXh1u
6,22520,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da clic aquí para vernos #EnVivo: https://t.co/UNxuKU6ikA https://t.co/qR2HRcByYy
7,40330,0.073,CiroGomezL,#CiroEnImagen Síguenos ahora en #TikTok a través de #ImagenNoticias Da clic aquí para vernos #EnVivo: https://t.co/mpQWecdvcT https://t.co/uCLEAdWWUA


  0.146073 seconds (166.64 k allocations: 3.814 MiB)


1,927,0.0,TelemundoNews,"El mensaje de #KamalaHarris a los migrantes desata críticas contra la vicepresidenta. ""Que una mujer hija de inmigrantes diga esto a personas desesperadas es simplemente desgarrador"", dijo una periodista hispana. https://t.co/upukAEeeal"
2,16191,0.041,CiroGomezL,"Mañana está citado a comparecer el ex director de #Pemex, Emilio Lozoya, en el Centro de Justicia Federal del Reclusorio Norte. Tendrá que presentar las pruebas para sustentar la denuncia contra 17 personas a quienes acusó de haber recibido sobornos de Odebrecht https://t.co/b1HuToW6ea"
3,37608,0.041,CiroGomezL,"María Herrera, originaria de #Michoacán y madre de 4 desaparecidos, fue al #Vaticano a entregarle al papa Francisco una carta en la que le pidió rezar por las madres que no encuentran a sus hijos y que haga un llamado a los gobiernos para exigir justicia. https://t.co/E9RlYVCptg"
4,28872,0.042,El_Universal_Mx,"#LaMañaneraDeAMLO 🔴 Vamos a esperar porque el gobernador dijo que se iba a actuar conforme a la ley, responde López Obrador al ser cuestionado si en la riña Querétaro vs Atlas hubo presencia de integrantes del crimen organizado. https://t.co/SaqeBQ1lwt"
5,32767,0.042,Milenio,Bukele afirma que sí recibió apoyo de Trump contra pandillas en #ElSalvador El presidente salvadoreño reclamó al gobierno de Biden de “apoyar a los pandilleros y sus libertades civiles” https://t.co/PkDZI943Ds https://t.co/0F02yDHvpa
6,35442,0.042,azucenau,"#AzucenaALas10 | El presidente López Obrador continúo con su gira por Centroamérica, visitó El Salvador; felicitó a Nayib Bukele por tener el apoyo de su pueblo y recordó ese país fue el primero que aplicó los programas ""sembrando vida"" y ""jóvenes, construyendo el futuro"" https://t.co/U065nPUebf"
7,4763,0.043,teleSURtv,"#ENVIVO| El Presidente de #Cuba, Miguel Díaz- Canel, hay una población joven e insatisfecha que en su legitimo derecho se alzaron para reclamar derechos, sin embargo, hubo una parte que su motivación es afectar a su pueblo. https://t.co/tqMKHfl0b4 https://t.co/rTG2ciMaMk"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.- Defina $H^{TF}_d$, donde $TF$ es la frecuencia de término normalizada.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
